In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [9]:
messages = ChatPromptTemplate.from_messages([SystemMessage(content=f"""
            You are a Tavily Search Query specialist. Follow the JSON schema below exactly:

            Rules:
            1. Generate Tavily DSL only (no natural language outside the JSON).
            2. Map the job description into five categories:
            • query1: recent developments
            • query2: recent news
            • query3:company profile
            • query4: key customers & partners
            • query5: culture & values
            3. Each value is a two‑element list:
            [<query string>, <one‑sentence rationale>]
            4. Use filters (source:, date:[now-30d TO now], site:…, etc.) where helpful.
            5. If information is missing in the JD, fall back sensibly
            (e.g. search for “employee testimonials”).
            6. Return **only** valid JSON.
        """
    )
    , HumanMessage(content="Hello World")])

In [6]:
input_message = ChatPromptTemplate.from_messages([HumanMessage(content="Hello World")])


In [11]:
messages.pretty_print()

================================ System Message ================================


            You are a Tavily Search Query specialist. Follow the JSON schema below exactly:

            Rules:
            1. Generate Tavily DSL only (no natural language outside the JSON).
            2. Map the job description into five categories:
            • query1: recent developments
            • query2: recent news
            • query3:company profile
            • query4: key customers & partners
            • query5: culture & values
            3. Each value is a two‑element list:
            [<query string>, <one‑sentence rationale>]
            4. Use filters (source:, date:[now-30d TO now], site:…, etc.) where helpful.
            5. If information is missing in the JD, fall back sensibly
            (e.g. search for “employee testimonials”).
            6. Return **only** valid JSON.
        

================================ Human Message =================================

Hello World

In [14]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

input_message = HumanMessagePromptTemplate.from_template("Below is the required job description and resume: {background_information}", input_variables=["background_information"])

In [17]:
input_message.format(background_information="This is Rishabh")

HumanMessage(content='Below is the required job description and resume: This is Rishabh', additional_kwargs={}, response_metadata={})

In [18]:
import re
from pathlib import Path
from typing import List

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import (
    MarkdownHeaderTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.schema import Document

In [29]:
def _collapse_ws(text: str) -> str:
    """Collapse stray whitespace but keep bullet breaks."""
    text = re.sub(r"\n\s*([•\-–])\s*", r"\n\1 ", text)
    return re.sub(r"[ \t\r\f\v]+", " ", text).replace(" \n", "\n").strip()


def _is_heading(line: str) -> bool:
    return (
        line.isupper()
        and len(line.split()) <= 5
        and not re.search(r"\d", line)
    )


def parse_resume(pdf_path: str | Path) -> List[Document]:
    """
    Load a single‑page résumé PDF → list[Document] chunks
    (≈400 chars, 50‑char overlap) with {source, section} metadata.
    """
    text = PyPDFLoader(str(pdf_path), extraction_mode="layout").load()[0].page_content
    print(text)
    text = _collapse_ws(text)

    # Tag headings with "###" so Markdown splitter can see them
    tagged_lines = [
        f"### {ln}" if _is_heading(ln) else ln
        for ln in text.splitlines()
    ]
    md_text = "\n".join(tagged_lines)

    if "###" in md_text:
        splitter = MarkdownHeaderTextSplitter(
            headers_to_split_on=[("###", "section")]
        )
        chunks = splitter.split_text(md_text)  # already returns Documents
    else:
        print(f"No headings found.")
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=400, chunk_overlap=50
        )
        chunks = [
            Document(page_content=chunk, metadata={})
            for chunk in splitter.split_text(md_text)
        ]

    # Attach metadata
    for doc in chunks:
        doc.metadata.setdefault("source", str(pdf_path))
        # section already present if header‑splitter was used
    return chunks


In [31]:
chunks = parse_resume("C:\\Users\\risha\\Downloads\\Rishabh_SDE_Resume.pdf")

Rishabh Aggarwal
                                      (602) 580-5734  •      raggar15@asu.edu  •       LinkedIn  •    Tempe, AZ
TECHNICAL       SKILLS
Programming Languages: Python, Java, JavaScript, Bash, HTML, CSS
Databases: SQL (PostgreSQL, MySQL, SQLite), NoSQL (MongoDB,  Redis, DynamoDB, Pinecone)
Frameworks/Tools: SpringBoot, React, JUnit, Node.js, RESTful APIs, Django, Kafka, Airﬂow, FastAPI, Pydantic, Tableau
DevOps/Cloud: AWS, GCP, GitHub Actions, Docker, Jenkins, Terraform, Kubernetes, MLFlow, GitLab
AI Tools/Frameworks: PyTorch, Tensorﬂow, scikit-learn, LangGraph, LangChain, LangSmith, ChatGPT
PROFESSIONAL EXPERIENCE
Amazon Inc, Tempe, AZ: Software Development Engineer | Seller Payment Services                                                Dec 2023 - Aug 2024
●   Established AWS Evidently        setup to handle 50K+ daily API requests           to new Lambda service using AWS CDK(TypeScript)
●   Added metrics      to monitor traﬃc and enhance service observability of the L

In [40]:
resume_text = ""
for chunk in chunks:
  print(f"Resume chunk: {chunk.page_content}")
  resume_text+= (chunk.page_content)

Resume chunk: Rishabh Aggarwal
(602) 580-5734 • raggar15@asu.edu • LinkedIn • Tempe, AZ
Resume chunk: Programming Languages: Python, Java, JavaScript, Bash, HTML, CSS
Databases: SQL (PostgreSQL, MySQL, SQLite), NoSQL (MongoDB, Redis, DynamoDB, Pinecone)
Frameworks/Tools: SpringBoot, React, JUnit, Node.js, RESTful APIs, Django, Kafka, Airﬂow, FastAPI, Pydantic, Tableau
DevOps/Cloud: AWS, GCP, GitHub Actions, Docker, Jenkins, Terraform, Kubernetes, MLFlow, GitLab
AI Tools/Frameworks: PyTorch, Tensorﬂow, scikit-learn, LangGraph, LangChain, LangSmith, ChatGPT
Resume chunk: Amazon Inc, Tempe, AZ: Software Development Engineer | Seller Payment Services Dec 2023 - Aug 2024
● Established AWS Evidently setup to handle 50K+ daily API requests to new Lambda service using AWS CDK(TypeScript)
● Added metrics to monitor traﬃc and enhance service observability of the Lambda service through CloudWatch logs
● Developed SNS Event Publishers in Java using Spring Boot to process 10K+ daily events in an ev

In [41]:
from pydantic import BaseModel, Field

class TavilyQuerySet(BaseModel):
    query1: tuple[str, str] = Field(
        ...,
        description="DSL for Recent Developments + 1‑sentence rationale",
    )
    query2: tuple[str, str] = Field(
        ...,
        description="DSL for Recent News + rationale",
    )
    query3: tuple[str, str]
    query4: tuple[str, str]
    query5: tuple[str, str]

In [42]:
from langchain.output_parsers import PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=TavilyQuerySet)

messages = SystemMessage(content=f"""
            You are a Tavily Search Query specialist. Follow the JSON schema below exactly:
            {parser.get_format_instructions()}

           
            Rules:
            1. Generate Tavily DSL only (no natural language outside the JSON).
            2. Map the job description into five categories:
            • query1: recent developments
            • query2: recent news
            • query3:company profile
            • query4: key customers & partners
            • query5: culture & values
            3. Each value is a two‑element list:
            [<query string>, <one‑sentence rationale>]
            4. Use filters (source:, date:[now-30d TO now], site:…, etc.) where helpful.
            5. If information is missing in the JD, fall back sensibly
            (e.g. search for “employee testimonials”).
            6. Return **only** valid JSON.
        """)

In [53]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"query1": {"description": "DSL for Recent Developments + 1‑sentence rationale", "maxItems": 2, "minItems": 2, "prefixItems": [{"type": "string"}, {"type": "string"}], "title": "Query1", "type": "array"}, "query2": {"description": "DSL for Recent News + rationale", "maxItems": 2, "minItems": 2, "prefixItems": [{"type": "string"}, {"type": "string"}], "title": "Query2", "type": "array"}, "query3": {"maxItems": 2, "minItems": 2, "prefixItems": [{"type": "string"}, {"type": "string"}], "title": "Query3", "type": "array"}, "

In [52]:
TavilyQuerySet.model_json_schema()

{'properties': {'query1': {'description': 'DSL for Recent Developments + 1‑sentence rationale',
   'maxItems': 2,
   'minItems': 2,
   'prefixItems': [{'type': 'string'}, {'type': 'string'}],
   'title': 'Query1',
   'type': 'array'},
  'query2': {'description': 'DSL for Recent News + rationale',
   'maxItems': 2,
   'minItems': 2,
   'prefixItems': [{'type': 'string'}, {'type': 'string'}],
   'title': 'Query2',
   'type': 'array'},
  'query3': {'maxItems': 2,
   'minItems': 2,
   'prefixItems': [{'type': 'string'}, {'type': 'string'}],
   'title': 'Query3',
   'type': 'array'},
  'query4': {'maxItems': 2,
   'minItems': 2,
   'prefixItems': [{'type': 'string'}, {'type': 'string'}],
   'title': 'Query4',
   'type': 'array'},
  'query5': {'maxItems': 2,
   'minItems': 2,
   'prefixItems': [{'type': 'string'}, {'type': 'string'}],
   'title': 'Query5',
   'type': 'array'}},
 'required': ['query1', 'query2', 'query3', 'query4', 'query5'],
 'title': 'TavilyQuerySet',
 'type': 'object'}

In [44]:
messages.pretty_print()

================================ System Message ================================


            You are a Tavily Search Query specialist. Follow the JSON schema below exactly:
            The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"query1": {"description": "DSL for Recent Developments + 1‑sentence rationale", "maxItems": 2, "minItems": 2, "prefixItems": [{"type": "string"}, {"type": "string"}], "title": "Query1", "type": "array"}, "query2": {"description": "DSL for Recent News + rationale", "maxItems": 2, "minItems": 2, "prefixItems": [{"type": "string"}, {"type": "strin

In [46]:
x = """properties": {"query1": [{"query": "Shalin Mehta AND \"Computational Microscopy Platform\"", "rationale": "Recent developments within the company"}, {"query": "Shalin Mehta AND \"Biohub SF\"", "rationale": "Recent developments within the company"}], "query2": [{"query": "Chan Zuckerberg Biohub - San Francisco AND recent news", "rationale": "Recent news about the company"}, {"query": "COVID-19 AND Chan Zuckerberg Biohub - San Francisco", "rationale": "Recent news about the company"}], "query3": [{"query": "Shalin Mehta AND \"role: Software Engineer\"", "rationale": "Information about the company that relates to the role"}, {"query": "Chan Zuckerberg Biohub - San Francisco AND \"team: Bioengineering\"", "rationale": "Information about the company that relates to the role"}], "query4": [{"query": "key customers: Chan Zuckerberg Biohub", "rationale": "Key customers & partners"}, {"query": "partners: Chan Zuckerberg Biohub SF", "rationale": "Key customers & partners"}], "query5": [{"query": "company culture: Chan Zuckerberg Biohub", "rationale": "Culture & values of the company"}, {"query": "values: Chan Zuckerberg Biohub", "rationale": "Culture & values of the company"}]}, "required": ["query1", "query2", "query3", "query4", "query5"]"""

In [49]:
print(x)

properties": {"query1": [{"query": "Shalin Mehta AND "Computational Microscopy Platform"", "rationale": "Recent developments within the company"}, {"query": "Shalin Mehta AND "Biohub SF"", "rationale": "Recent developments within the company"}], "query2": [{"query": "Chan Zuckerberg Biohub - San Francisco AND recent news", "rationale": "Recent news about the company"}, {"query": "COVID-19 AND Chan Zuckerberg Biohub - San Francisco", "rationale": "Recent news about the company"}], "query3": [{"query": "Shalin Mehta AND "role: Software Engineer"", "rationale": "Information about the company that relates to the role"}, {"query": "Chan Zuckerberg Biohub - San Francisco AND "team: Bioengineering"", "rationale": "Information about the company that relates to the role"}], "query4": [{"query": "key customers: Chan Zuckerberg Biohub", "rationale": "Key customers & partners"}, {"query": "partners: Chan Zuckerberg Biohub SF", "rationale": "Key customers & partners"}], "query5": [{"query": "compan

In [54]:
from langchain_core.prompts import (
    PromptTemplate,
)

In [ ]:
prompt = PromptTemplate.from_template("Below is the required job description and resume: {background_information}", input_variables=["background_information"])

In [55]:
x = ('query1', ('recent developments within the company', 'The Associate Software engineer will build open source software tools for managing and processing 10-100 terabyte-scale datasets.'))

In [61]:
keys = ('q', ('y', 'z'))

dict_x = dict(zip(keys, x))

In [63]:
dict_x[('y', 'z')]

('recent developments within the company',
 'The Associate Software engineer will build open source software tools for managing and processing 10-100 terabyte-scale datasets.')

In [ ]:
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser, RetryOutputParser
base_parser = PydanticOutputParser(pydantic_object=TavilyQuerySet)



In [1]:
format_instructions = parser.get_format_instructions()
from ollama import chat


tavily_role_messages = SystemMessage(content=
            f"""
            When you reply, output **only** valid JSON that can be parsed
            into the Pydantic model shown below. Do **not** wrap it in "properties"
            or "required".:
            
            ------------------------------------------------


            {format_instructions}

            
           -------------------------------------------------

            Rules:
            1. Generate Tavily DSL only (no natural language outside the JSON).
            2. Map the job description into five categories:
            • query1: recent developments within the company
            • query2: recent news about the company
            • query3: information about the company that relates to the role
            • query4: key customers & partners
            • query5: culture & values of the company
            3. Each value is a two‑element list:
            [<query string>, <one‑sentence rationale>]
            4. If information is missing in the JD, fall back sensibly
            (e.g. search for “employee testimonials”).
            5. Return **only** valid JSON that matches the schema exactly. No other fields
            """)


response = chat(
    messages=[{
        tavily_role_messages,
        input_message}
      ],
    model='llama3.2:latest',
    format=TavilyQuerySet.model_json_schema(),
    )

NameError: name 'parser' is not defined

In [2]:
p = ('query1', ['Recent developments within the company using computational microscopy platform', 'This project will require working on microscopes in a BSL-2 imaging laboratory'])

In [3]:
p[1][0]

'Recent developments within the company using computational microscopy platform'

In [6]:
COVER_LETTER_PROMPT = SystemMessage(content="""You are my dedicated assistant for writing job application content, including cover letters, LinkedIn outreach messages, and responses to job-specific questions (e.g., experience, culture fit, or motivation).

Your goal is to generate content that:
1. Reflects **my personality**, tone, and authentic voice, based on examples I provide.
2. Matches **my knowledge, experience, and interests**, which I’ll also share or update as needed.
3. Adopts **my writing style and energy** (e.g., grounded, confident, thoughtful—but not overly polished or generic).
4. Embeds **genuine enthusiasm or alignment** with the company or role, without sounding performative.
5. Avoids filler, clichés, or overused corporate phrases—keep it **authentic and specific**.
6. Learns over time by asking me relevant clarifying questions when needed (e.g., change in tone, new experience, updates to goals).
7. Balances job description alignment with personal storytelling, roughly in a 75:25 ratio.
8. Keeps outputs **concise** and within any given word or character limits.""")


In [7]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)

FirstDraftGenerationPromptTemplate = ChatPromptTemplate.from_messages([COVER_LETTER_PROMPT])

In [8]:
FirstDraftGenerationPromptTemplate

ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[SystemMessage(content='You are my dedicated assistant for writing job application content, including cover letters, LinkedIn outreach messages, and responses to job-specific questions (e.g., experience, culture fit, or motivation).\n\nYour goal is to generate content that:\n1. Reflects **my personality**, tone, and authentic voice, based on examples I provide.\n2. Matches **my knowledge, experience, and interests**, which I’ll also share or update as needed.\n3. Adopts **my writing style and energy** (e.g., grounded, confident, thoughtful—but not overly polished or generic).\n4. Embeds **genuine enthusiasm or alignment** with the company or role, without sounding performative.\n5. Avoids filler, clichés, or overused corporate phrases—keep it **authentic and specific**.\n6. Learns over time by asking me relevant clarifying questions when needed (e.g., change in tone, new experience, updates to goals).

In [ ]:
current_application_session  = "Heello World"
company_research_data = "Company Research Data"

In [10]:
CurrentSessionContextMessage = HumanMessagePromptTemplate.from_template(
            """
            # Resume and Job Description
            {current_job_role}

            # Company Information
            {company_research_data}

            Create a cover letter that highlights the match between my qualifications and the job requirements.
            """,
            input_variables=["current_job_role",
                        "company_research_data"])

In [17]:
FirstDraftGenerationPromptTemplate.append(CurrentSessionContextMessage)

In [18]:
chain = (
            ({"current_job_role": lambda x: x["current_job_role"],
              "company_research_data": lambda x: x["company_research_data"]})
            | FirstDraftGenerationPromptTemplate
    )

In [19]:
chain

{
  current_job_role: RunnableLambda(...),
  company_research_data: RunnableLambda(...)
}
| ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[SystemMessage(content='You are my dedicated assistant for writing job application content, including cover letters, LinkedIn outreach messages, and responses to job-specific questions (e.g., experience, culture fit, or motivation).\n\nYour goal is to generate content that:\n1. Reflects **my personality**, tone, and authentic voice, based on examples I provide.\n2. Matches **my knowledge, experience, and interests**, which I’ll also share or update as needed.\n3. Adopts **my writing style and energy** (e.g., grounded, confident, thoughtful—but not overly polished or generic).\n4. Embeds **genuine enthusiasm or alignment** with the company or role, without sounding performative.\n5. Avoids filler, clichés, or overused corporate phrases—keep it **authentic and specific**.\n6. Learns over time by asking me relevan

In [ ]:
from job_writer.utils.llm_client import LLMClient

LLM = LLMClient()
llm = LLMClient().get_llm()

ModuleNotFoundError: No module named 'utils'

In [ ]:
from job_writer.tools.TavilySearch import search_company

# Test job description
test_job = """
Software Engineer - Backend
OpenAI

We are looking for experienced backend engineers to join our team. Our ideal candidate will have experience with one or more of the following technologies: Python, Java, C++. 

Responsibilities:
- Design and implement scalable and efficient backend systems
- Write clean, maintainable code
- Work with cross-functional teams

Requirements:
- Strong proficiency in one or more programming languages
- Strong understanding of software design patterns and principles
- Experience with distributed systems
"""

# Test the search_company function
results = search_company(test_job)
for query_key, data in results.items():
    print(f"\n{query_key}:")
    print(f"Query: {data['query']}")
    print(f"Rationale: {data['rationale']}")
    if data['results']:
        print(f"First result: {data['results'][0][:200]}...")
    else:
        print("No results found")


In [1]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)

In [2]:
from job_writer.prompts.templates import (
    TAVILY_QUERY_PROMPT
)

In [3]:
tavily_search_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content=TAVILY_QUERY_PROMPT),
    HumanMessage(
        "Below is the required job description and resume: {background_information}",
        input_variables=["background_information"]
    )
])

In [5]:
job_description = """
Software Engineer - Backend
OpenAI

We are looking for experienced backend engineers to join our team. Our ideal candidate will have experience with one or more of the following technologies: Python, Java, C++. 

Responsibilities:
- Design and implement scalable and efficient backend systems
- Write clean, maintainable code
- Work with cross-functional teams

Requirements:
- Strong proficiency in one or more programming languages
- Strong understanding of software design patterns and principles
- Experience with distributed systems
"""

In [6]:
tavily_search_prompt.format(background_information=job_description)

'System: \n<Background>\nSINCE THE USER IS APPPLYING FOR A JOB, THE QUERIES SHOULD BE WRITTEN IN A WAY THAT RESULST IN RELEVANT INFORMATION ABOUT THE COMPANY. THIS WILL HELP THE USER WRITE A MORE PERSONALIZED AND RELEVANT APPLICATION.\n\nCategory mapping (remember this!):\n    query1 : recent developments\n    query2 : recent news\n    query3 : role-related info\n    query4 : key customers & partners  \n    query5 : culture & values\n\nNote: The above are just categories. The queries should be written in a way that results in relevant information about the company. Must include the company name in the query to ensure results have a higher confidence.\n</Background>\n\n<Instructions>\n    1. Each array must contain **exactly two** strings: [search_query, one_sentence_rationale]  \n    2. If data is missing, craft a sensible fallback query; never return an empty array.  \n    3. If the employer name cannot be found, use `"UNKNOWN"`.  \n    4. Escape JSON only where required.\n    5. Quer

In [8]:
from job_writer.utils.llm_client import LLMClient

LLM = LLMClient()
llm = LLMClient().get_llm()

Initializing LLM with model llama3.2:latest and provider ollama in c:\users\risha\python-dir\knowledgebase\job_writer\utils\llm_client.py
Initializing LLM with model llama3.2:latest and provider ollama in c:\users\risha\python-dir\knowledgebase\job_writer\utils\llm_client.py
